In [1]:
library(tidyverse)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
library(brms)
library(ape)


Loading required package: Rcpp

Loading 'brms' package (version 2.14.4). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar




In [3]:
d <- read_csv("../data/data_cues.txt")


── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Glottocode = col_character(),
  Language = col_character(),
  Genus = col_character(),
  Case_Marking = col_double(),
  Tight_Semantics = col_double(),
  Rigid_Order = col_double(),
  Verb_Middle = col_double()
)



In [4]:
head(d)

Glottocode,Language,Genus,Case_Marking,Tight_Semantics,Rigid_Order,Verb_Middle
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
stan1318,Arabic,Semitic,0.036,0.218,0.655,0.391
bulg1262,Bulgarian,Slavic,0.028,0.144,0.782,0.966
croa1245,Croatian,Slavic,0.415,0.147,0.414,0.900
czec1258,Czech,Slavic,0.525,0.172,0.240,0.818
dani1285,Danish,Germanic,0.000,0.208,0.926,0.912
dutc1256,Dutch,Germanic,0.000,0.214,0.817,0.462


In [5]:
d %>%
    mutate(
        x1 = scale(Case_Marking), 
        x2 = scale(Tight_Semantics),
        x3 = scale(Rigid_Order),
        x4 = scale(Verb_Middle)
    ) -> d
head(d)

Glottocode,Language,Genus,Case_Marking,Tight_Semantics,Rigid_Order,Verb_Middle,x1,x2,x3,x4
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
stan1318,Arabic,Semitic,0.036,0.218,0.655,0.391,-0.7628649,0.826553846,0.09797316,-0.7625740
bulg1262,Bulgarian,Slavic,0.028,0.144,0.782,0.966,-0.7927616,-0.515764763,0.55542136,0.7687882
croa1245,Croatian,Slavic,0.415,0.147,0.414,0.900,0.6534927,-0.461346441,-0.77009783,0.5930144
czec1258,Czech,Slavic,0.525,0.172,0.240,0.818,1.0645728,-0.007860424,-1.39683789,0.3746289
dani1285,Danish,Germanic,0.000,0.208,0.926,0.912,-0.8974002,0.645159439,1.07410278,0.6249733
dutc1256,Dutch,Germanic,0.000,0.214,0.817,0.462,-0.8974002,0.753996083,0.68148976,-0.5734840


In [6]:
frm = formula(x1 ~ x2)

In [7]:
summary(lm(frm, d))


Call:
lm(formula = frm, data = d)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.1594 -0.6078 -0.3937  0.6625  2.3716 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)  
(Intercept) -9.516e-17  1.742e-01    0.00     1.00  
x2           3.474e-01  1.772e-01    1.96     0.06 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9543 on 28 degrees of freedom
Multiple R-squared:  0.1207,	Adjusted R-squared:  0.0893 
F-statistic: 3.844 on 1 and 28 DF,  p-value: 0.05996


In [8]:
phylo <- read.nexus("../data/mcc.tre")

In [9]:
A <- 10*ape::vcv.phylo(phylo)

In [10]:
model1_2 <- brm(
    x1 ~ x2 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 1), "b"),
        prior(normal(0, 1), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model1_2)

Compiling Stan program...

Start sampling

Warning message:
“There were 9 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess”
Warning message:
“There were 9 divergent transitions af

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x1 ~ x2 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.51      0.13     0.27     0.76 1.02      177      713

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     0.03      0.33    -0.64     0.68 1.00      981     1032
x2            0.22      0.16    -0.09     0.55 1.00     1352     2089

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.44      0.20     0.07     0.85 1.04       85       70

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 

In [11]:
model1_3 <- brm(
    x1 ~ x3 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 10), "b"),
        prior(normal(0, 50), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model1_3)

Compiling Stan program...

Start sampling

Warning message:
“There were 26 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 19 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.06, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the cha

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x1 ~ x3 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.40      0.09     0.22     0.56 1.03      133      267

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     0.05      0.25    -0.43     0.57 1.01      695     1019
x3           -0.78      0.14    -1.05    -0.50 1.01      775     1494

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.20      0.15     0.02     0.56 1.05       50      113

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 

In [12]:
model1_4 <- brm(
    x1 ~ x4 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 10), "b"),
        prior(normal(0, 50), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model1_4)

Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“There were 2 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 3 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.07, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and med

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x1 ~ x4 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.54      0.11     0.34     0.77 1.02      225     1192

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept    -0.12      0.38    -0.84     0.63 1.01      960      956
x4           -0.37      0.22    -0.80     0.07 1.01      825     1328

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.35      0.18     0.07     0.73 1.06       53      119

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 

In [13]:
model2_3 <- brm(
    x2 ~ x3 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 10), "b"),
        prior(normal(0, 50), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model2_3)

Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“There were 9 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 160 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.05, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and m

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x2 ~ x3 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.50      0.24     0.04     0.90 1.04      125      790

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     0.03      0.38    -0.72     0.87 1.01      689      680
x3           -0.18      0.24    -0.65     0.29 1.00     1151     1002

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.65      0.32     0.09     1.21 1.05       84      109

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 

In [14]:
model2_4 <- brm(
    x2 ~ x4 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 10), "b"),
        prior(normal(0, 50), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model2_4)

Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“There were 14 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.05, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, indica

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x2 ~ x4 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.51      0.22     0.06     0.87 1.04      120      697

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept    -0.15      0.39    -0.97     0.60 1.00      789      802
x4           -0.43      0.23    -0.90     0.02 1.00     1239     1645

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.57      0.30     0.07     1.12 1.05       77       91

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 

In [15]:
model3_4 <- brm(
    x3 ~ x4 + (1|gr(Language, cov = A)),
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 10), "b"),
        prior(normal(0, 50), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),    
    iter=2000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
summary(model3_4)

Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“There were 13 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1716 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“The largest R-hat is 1.13, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and

 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x3 ~ x4 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
Samples: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup samples = 4000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.56      0.10     0.37     0.79 1.05       65      225

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     0.10      0.37    -0.62     0.86 1.01      351      481
x4            0.34      0.22    -0.09     0.79 1.03      235      303

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.27      0.18     0.01     0.66 1.13       23       22

Samples were drawn using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the 